In [25]:
from serde import serde
from serde.json import to_json, from_json
from dataclasses import dataclass
from time import sleep
from scrapping import Voiture
import sklearn
import numpy as np


# Récuperation du json et conversion en quelque chose d'utilisable

In [26]:
with open("donnees.json", "r") as fichier:
    contenu_fichier = fichier.read()

In [27]:
reconstitution = from_json(list[Voiture],contenu_fichier)



In [28]:
reconstitution[0]

Voiture(modele='308', carburant='Diesel', prix=20290, kilometrage=75967, garantie_kilometrage='non garanti', boite_de_vitesse='Automatique', transmission=2, couleur='Blanc', garantie='SPOTICAR PREMIUM', date_mise_circulation=2019, puissance=130, silhouette='Berline', nb_places=5, utilisation_prec='Ex-Loueur', puissance_fiscale=6, critair=2, ptac=1890, nb_portes=5)

# Conversion en base de données panda et nettoyage

In [29]:
import pandas as pd

In [30]:
df = pd.DataFrame(reconstitution)


In [31]:
df.describe()

,prix,kilometrage,date_mise_circulation
count,600.000000,600.000000,600.000000
mean,26750.178333,28932.145000,2020.473333
std,8835.590982,29263.082939,1.738100
min,7490.000000,3.000000,2011.000000
25%,19721.750000,7000.000000,2019.000000
50%,26240.000000,19521.000000,2021.000000
75%,33524.750000,41282.500000,2022.000000
max,59980.000000,176600.000000,2022.000000


In [32]:
df.head()


,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes
0,308,Diesel,20290,75967,non garanti,Automatique,2,Blanc,SPOTICAR PREMIUM,2019,130,Berline,5,Ex-Loueur,6,2,1890,5
1,3008,Diesel,31900,29066,non garanti,Manuelle,NA,Gris,SPOTICAR PREMIUM,2021,130,SUV-4x4,5,NA,7,2,NA,5
2,3008,Essence,33920,31997,non garanti,Automatique,NA,Gris,SPOTICAR PREMIUM,2021,130,SUV-4x4,5,NA,7,1,NA,5
3,2008,Essence,21980,40952,non garanti,Manuelle,NA,Gris,SPOTICAR PREMIUM,2021,100,SUV-4x4,5,NA,5,1,NA,5
4,2008,Essence,22990,7901,non garanti,Manuelle,NA,Gris,SPOTICAR PREMIUM,2021,100,SUV-4x4,5,Ex-Particulier,5,1,NA,5


In [33]:
for i in df.columns:
    col_selec = f"df.{i}.unique()"
    
    print(f"{i} : {eval(col_selec)} ")
# retourne toutes les modalités pour chaque variable

modele : ['308' '3008' '2008' '208' '108' '308 SW' '5008' '508' 'RIFTER' 'BOXER'
 '508 SW' 'TRAVELLER' 'PARTNER' 'EXPERT' '4008'] 
carburant : ['Diesel' 'Essence' 'Electrique' 'Hybride rechargeable'] 
prix : [20290 31900 33920 21980 22990 21990 34390 17890 32990 20490 18990 11990
 26290 33490 17490 32490 25990 26590 22490 17690 34490 36990 38790 32590
 28990 25290 16990 22190 23980 33790 38490 22390 26990 29990 38980 31690
 24590 35990 38290 28790 26190 23790 26299 21490 31290 35690 22290 46790
 15490 28980 33091 30790 40490 28490 36390 17990 39790 23220 59980 38990
 24790 23290 32480 20480 34990 27790 23990 15480 23390 52890 29290 27590
 27990 26490 15290 27690 39990 42580 36490 32390 28690 32300 24990 19290
 42590 46290 31580 52190 37990 28590 39390 39690 38280 24490 23150 26390
 37180 21590 35299 38499 37099 24199 42199 14790 14799 37490 39631 47490
 37599 14590 33990 28390 16490 18480 17980 22980 39290 22180 27980 12480
 19280 33480 12280 22280 33980 31780 40980 24280 15380 18580 3

## Remarques sur les variables :

- modele : enlever les espaces, ou supprimer les variables peu occurentes (<=1)
- carburant : même remarque
- même remarque pour toutes les variables textuelles
- utilisation précédente : regrouper par pro/loueur/part/ ne sait pas ?

In [34]:
from sklearn.base import TransformerMixin

df = df.replace('NA', np.NaN)

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)
df
df = DataFrameImputer().fit_transform(df)

## Création de dummys


In [35]:
df.head()

,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes
0,308,Diesel,20290,75967,non garanti,Automatique,2.000000,Blanc,SPOTICAR PREMIUM,2019,130.0,Berline,5.0,Ex-Loueur,6.0,2.0,1890.000000,5.0
1,3008,Diesel,31900,29066,non garanti,Manuelle,2.014925,Gris,SPOTICAR PREMIUM,2021,130.0,SUV-4x4,5.0,Ex-Loueur,7.0,2.0,1874.729367,5.0
2,3008,Essence,33920,31997,non garanti,Automatique,2.014925,Gris,SPOTICAR PREMIUM,2021,130.0,SUV-4x4,5.0,Ex-Loueur,7.0,1.0,1874.729367,5.0
3,2008,Essence,21980,40952,non garanti,Manuelle,2.014925,Gris,SPOTICAR PREMIUM,2021,100.0,SUV-4x4,5.0,Ex-Loueur,5.0,1.0,1874.729367,5.0
4,2008,Essence,22990,7901,non garanti,Manuelle,2.014925,Gris,SPOTICAR PREMIUM,2021,100.0,SUV-4x4,5.0,Ex-Particulier,5.0,1.0,1874.729367,5.0


In [36]:
df.dtypes


modele                    object
carburant                 object
prix                       int64
kilometrage                int64
garantie_kilometrage      object
boite_de_vitesse          object
transmission             float64
couleur                   object
garantie                  object
date_mise_circulation      int64
puissance                float64
silhouette                object
nb_places                float64
utilisation_prec          object
puissance_fiscale        float64
critair                  float64
ptac                     float64
nb_portes                float64
dtype: object

In [37]:
df.ptac

0      1890.000000
1      1874.729367
2      1874.729367
3      1874.729367
4      1874.729367
          ...     
595    1940.000000
596    1770.000000
597    2000.000000
598    1874.729367
599    2790.000000
Name: ptac, Length: 600, dtype: float64

In [38]:
df_mod = pd.get_dummies(df["modele"])
df_garkil = pd.get_dummies(df["garantie_kilometrage"])
df_carbu = pd.get_dummies(df["carburant"])
df_bdv = pd.get_dummies(df["boite_de_vitesse"])
df_couleur = pd.get_dummies(df["couleur"])
df_sil = pd.get_dummies(df["silhouette"])
df_trans = pd.get_dummies(df["transmission"])
df_gar = pd.get_dummies(df["garantie"])
df_utilprec = pd.get_dummies(df["utilisation_prec"])

df = pd.concat([df, df_mod, df_garkil, df_carbu, df_bdv, df_couleur, df_sil,df_trans, df_gar, df_utilprec], axis = 1)

del df['modele']
del df['garantie_kilometrage']
del df['carburant']
del df['boite_de_vitesse']
del df['couleur']
del df['silhouette']
del df['transmission']
del df['garantie']
del df['utilisation_prec']

In [39]:
df.convert_dtypes().dtypes

prix                       Int64
kilometrage                Int64
date_mise_circulation      Int64
puissance                Float64
nb_places                Float64
                          ...   
Ex-Import                  UInt8
Ex-Loueur                  UInt8
Ex-Particulier             UInt8
Ex-Société                 UInt8
Véhicule de direction      UInt8
Length: 61, dtype: object

## Passage sous numpy + ML

In [40]:
y = df['prix'].to_numpy()
X = df.loc[:, df.columns != 'prix'].to_numpy() # tout sauf le prix

In [41]:
type(y), y.shape

(numpy.ndarray, (600,))

In [42]:
type(X), X.shape

(numpy.ndarray, (600, 60))

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import OneHotEncoder

In [44]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y)

In [45]:
en = ElasticNet()
en_gs = GridSearchCV(
    en,
    {
        "alpha": [2 ** p  for p in range(-6, 6)],
        "l1_ratio": (0.01, 0.25, 0.5, 0.75, 1),
    }
)
en_gs.fit(X_tr, y_tr) #problème vient du fit
en_df = pd.DataFrame(en_gs.cv_results_)
en_df.head()
        

c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.134e+08, tolerance: 2.795e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.766e+08, tolerance: 2.695e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.020853,0.006565,0.000199,0.000399,0.015625,0.01,"{'alpha': 0.015625, 'l1_ratio': 0.01}",0.933275,0.936201,0.927212,0.926809,0.904825,0.925665,0.011017,1
1,0.017548,0.001854,0.000599,0.000489,0.015625,0.25,"{'alpha': 0.015625, 'l1_ratio': 0.25}",0.933836,0.936086,0.927373,0.925678,0.905010,0.925597,0.010999,4
2,0.025418,0.006106,0.000206,0.000411,0.015625,0.5,"{'alpha': 0.015625, 'l1_ratio': 0.5}",0.934318,0.935856,0.927499,0.924229,0.904868,0.925354,0.011101,6
3,0.039529,0.006709,0.000161,0.000322,0.015625,0.75,"{'alpha': 0.015625, 'l1_ratio': 0.75}",0.934457,0.935572,0.927515,0.922636,0.903952,0.924826,0.011454,11
4,0.033407,0.024662,0.000399,0.000489,0.015625,1,"{'alpha': 0.015625, 'l1_ratio': 1}",0.929552,0.936029,0.925801,0.923090,0.903765,0.923648,0.010848,14


In [46]:
en_gs.best_params_, en_gs.best_score_

({'alpha': 0.015625, 'l1_ratio': 0.01}, 0.9256647102742164)

In [47]:
from sklearn.neighbors import KNeighborsRegressor

KNN

In [48]:
knr = KNeighborsRegressor()
knr_gs = GridSearchCV(
    knr,
    {
        "n_neighbors": range(5, 15),
        "weights": ("uniform", "distance"),
    }
)

In [49]:
knr_gs.fit(X_tr, y_tr)
knr_df = pd.DataFrame(knr_gs.cv_results_)
knr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000199,0.000398,0.036334,0.052924,5,uniform,"{'n_neighbors': 5, 'weights': 'uniform'}",0.393686,0.545231,0.486610,0.354025,0.513449,0.458600,0.072729,11
1,0.000200,0.000399,0.010156,0.000728,5,distance,"{'n_neighbors': 5, 'weights': 'distance'}",0.498206,0.630906,0.506426,0.417385,0.545511,0.519687,0.069483,1
2,0.000399,0.000489,0.007598,0.000874,6,uniform,"{'n_neighbors': 6, 'weights': 'uniform'}",0.344550,0.500736,0.501512,0.321497,0.509074,0.435474,0.084018,12
3,0.000598,0.000489,0.009175,0.000977,6,distance,"{'n_neighbors': 6, 'weights': 'distance'}",0.469705,0.606986,0.522388,0.399321,0.539047,0.507490,0.069635,2
4,0.000199,0.000399,0.007386,0.000800,7,uniform,"{'n_neighbors': 7, 'weights': 'uniform'}",0.350733,0.473300,0.496350,0.318003,0.513581,0.430394,0.080111,13


In [50]:
knr_gs.best_params_, knr_gs.best_score_

({'n_neighbors': 5, 'weights': 'distance'}, 0.5196868614199994)

In [51]:
from sklearn.ensemble import RandomForestRegressor

In [52]:
rfr = RandomForestRegressor()
rfr_gs = GridSearchCV(
    rfr,
    {   
        "n_estimators": (8 , 16, 32, 64, 128, 256),
    }
)

In [53]:
rfr_gs.fit(X_tr, y_tr)
rfr_df = pd.DataFrame(rfr_gs.cv_results_)
rfr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.029512,0.005615,0.001004,0.000012,8,{'n_estimators': 8},0.925824,0.915519,0.870399,0.952284,0.794774,0.891760,0.055228,5
1,0.054459,0.004882,0.001597,0.000490,16,{'n_estimators': 16},0.889649,0.929259,0.880166,0.945556,0.802580,0.889442,0.049730,6
2,0.097370,0.007137,0.002601,0.000494,32,{'n_estimators': 32},0.903029,0.924574,0.892092,0.948288,0.808917,0.895380,0.047322,4
3,0.203687,0.014108,0.004673,0.000568,64,{'n_estimators': 64},0.910765,0.930842,0.887422,0.957805,0.812999,0.899966,0.049267,1
4,0.382589,0.016073,0.009377,0.001012,128,{'n_estimators': 128},0.910221,0.930904,0.888337,0.950505,0.811678,0.898329,0.048027,3


In [54]:
rfr_gs.best_params_, rfr_gs.best_score_

({'n_estimators': 64}, 0.8999664930794232)

In [55]:
from sklearn.svm import SVR

In [56]:
svr = SVR()
svr_gs = GridSearchCV(
    svr,
    {
        "C": (0.1, 1.0, 10),
        "epsilon": (0.1, 1.0, 10),
    }
)
svr_gs.fit(X_tr, y_tr)

svr_df = pd.DataFrame(svr_gs.cv_results_)
svr_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009191,0.001461,0.004737,0.000737,0.1,0.1,"{'C': 0.1, 'epsilon': 0.1}",-0.017628,-0.001282,-0.001243,-0.021165,0.000179,-0.008228,0.009203,8
1,0.008248,0.000380,0.004576,0.000475,0.1,1.0,"{'C': 0.1, 'epsilon': 1.0}",-0.017628,-0.001282,-0.001243,-0.021150,0.000179,-0.008225,0.009198,7
2,0.008285,0.000386,0.004403,0.000464,0.1,10,"{'C': 0.1, 'epsilon': 10}",-0.017903,-0.001283,-0.001243,-0.021420,0.000179,-0.008334,0.009330,9
3,0.008665,0.000552,0.004309,0.000403,1.0,0.1,"{'C': 1.0, 'epsilon': 0.1}",-0.014538,0.002247,0.003624,-0.018156,0.004408,-0.004483,0.009779,6
4,0.008314,0.000402,0.003975,0.000633,1.0,1.0,"{'C': 1.0, 'epsilon': 1.0}",-0.014538,0.002247,0.003624,-0.018155,0.004408,-0.004483,0.009778,5


In [57]:
svr_gs.best_params_, svr_gs.best_score_

({'C': 10, 'epsilon': 0.1}, 0.030575584062012596)

In [58]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

In [59]:
pl = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("support_vecteurs", SVR()),
    ]
)

In [60]:
pl_gs = GridSearchCV(
    pl,
    {
        "support_vecteurs__C": (0.1, 1.0, 10),
        "support_vecteurs__epsilon": (0.1, 1.0, 10),
    }
)

In [61]:
pl_gs.fit(X_tr, y_tr)

pl_df = pd.DataFrame(pl_gs.cv_results_)
pl_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_support_vecteurs__C,param_support_vecteurs__epsilon,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.009014,0.000625,0.004991,0.001094,0.1,0.1,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.017650,-0.001316,-0.001478,-0.021196,0.000009,-0.008326,0.009144,7
1,0.008586,0.000508,0.004996,0.000645,0.1,1.0,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.017665,-0.001316,-0.001478,-0.021207,0.000009,-0.008331,0.009150,8
2,0.008594,0.000503,0.004310,0.000570,0.1,10,"{'support_vecteurs__C': 0.1, 'support_vecteurs...",-0.017948,-0.001316,-0.001478,-0.021509,0.000010,-0.008448,0.009293,9
3,0.008328,0.000449,0.005052,0.000642,1.0,0.1,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",-0.014730,0.001943,0.001301,-0.018238,0.002741,-0.005397,0.009132,4
4,0.008629,0.000536,0.004254,0.000523,1.0,1.0,"{'support_vecteurs__C': 1.0, 'support_vecteurs...",-0.014730,0.001943,0.001301,-0.018238,0.002741,-0.005397,0.009132,4


In [62]:
pl_gs.best_params_, pl_gs.best_score_

({'support_vecteurs__C': 10, 'support_vecteurs__epsilon': 0.1},
 0.02339087365131003)

In [63]:
from sklearn.neural_network import MLPRegressor
import numpy as np

In [64]:
pln = Pipeline(
    [
        ("mise_echelle", MinMaxScaler()),
        ("neurones", MLPRegressor()),
    ]
)

In [65]:
pln_gs = GridSearchCV(
    pln,
    {
        "neurones__alpha": 10.0 ** -np.arange(1, 7),
        'neurones__hidden_layer_sizes': ((25,), (50, ), (100,), (20, 20)),
    }
)

In [66]:
pln_gs.fit(X_tr, y_tr)

pln_df = pd.DataFrame(pln_gs.cv_results_)
pln_df.head()

c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the opti

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_neurones__alpha,param_neurones__hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.287474,0.011993,0.000796,0.000398,0.1,"(25,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-9.973693,-8.347866,-9.123720,-9.941710,-8.830754,-9.243549,0.633648,19
1,0.537941,0.039821,0.000994,0.000006,0.1,"(50,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-9.911501,-8.290616,-9.055528,-9.885108,-8.800340,-9.188618,0.629689,13
2,0.766088,0.043108,0.000999,0.000633,0.1,"(100,)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-9.773714,-8.165951,-8.948731,-9.739247,-8.682258,-9.061980,0.620506,7
3,0.208718,0.029331,0.000598,0.000488,0.1,"(20, 20)","{'neurones__alpha': 0.1, 'neurones__hidden_lay...",-9.175563,-7.663906,-8.346570,-9.223261,-8.331800,-8.548220,0.586302,5
4,0.324173,0.039974,0.001194,0.000400,0.01,"(25,)","{'neurones__alpha': 0.01, 'neurones__hidden_la...",-9.984055,-8.340601,-9.113717,-9.955898,-8.877568,-9.254368,0.635817,22


In [67]:
pln_gs.best_params_

pln_gs.best_score_

-8.471098568265981

In [68]:
pln_gs.best_params_

{'neurones__alpha': 0.0001, 'neurones__hidden_layer_sizes': (20, 20)}

In [69]:
essai = en_gs.best_estimator_

In [70]:
prix_prédits = essai.predict(X)

In [71]:
prix_reels = y

In [72]:
prix_reels,prix_prédits

(array([20290, 31900, 33920, 21980, 22990, 21990, 34390, 17890, 32990,
        20490, 18990, 11990, 26290, 33490, 17490, 32490, 25990, 26590,
        26290, 22490, 17690, 34490, 36990, 38790, 18990, 32590, 28990,
        25290, 16990, 22190, 23980, 33790, 38490, 22390, 22490, 26990,
        29990, 38980, 25990, 31690, 24590, 35990, 38290, 28790, 32490,
        26190, 34490, 23790, 26299, 38290, 35990, 21490, 25990, 16990,
        31290, 35690, 22290, 46790, 15490, 28980, 21990, 21990, 33091,
        30790, 40490, 28490, 36390, 17990, 39790, 17990, 38490, 23220,
        22190, 59980, 38990, 24790, 23290, 32480, 20480, 34990, 24590,
        27790, 38980, 28490, 23990, 26590, 22190, 15480, 23390, 34490,
        52890, 29290, 27590, 27990, 26490, 34490, 32490, 15290, 35690,
        27690, 35690, 39990, 42580, 36490, 26190, 32390, 35990, 28690,
        32300, 28990, 24990, 26590, 19290, 29990, 42590, 46290, 31580,
        52190, 42590, 37990, 28590, 19290, 39390, 39990, 39690, 26990,
      

In [73]:
len(prix_prédits)

600

In [74]:
len(prix_reels)

600

In [75]:
prix_prédits

array([21670.75417329, 29503.83450997, 30052.10488689, 21985.13263657,
       25035.29002084, 24031.26842491, 35138.66333235, 18879.10643096,
       33459.08896505, 18530.96065918, 20797.40367146, 16770.56279792,
       26881.12719785, 33949.54093727, 18527.32932825, 32327.92217995,
       24753.46599049, 26759.85907048, 29784.61963093, 20982.08772809,
       18699.59615547, 34920.56150118, 39038.17610192, 35807.70176349,
       20514.29020011, 32300.17863088, 30111.76945882, 26704.62146986,
       17378.87062172, 23379.10869677, 25037.37271694, 32031.89499949,
       36045.76686086, 22448.58221417, 22659.96540535, 28853.50260362,
       28744.90105364, 36303.10719296, 27921.68612108, 29702.23231896,
       25438.32935863, 34296.42794268, 36662.73864068, 31912.2475878 ,
       31413.03028616, 27482.47589817, 33761.72227867, 23689.67429859,
       26926.05372372, 36158.49789726, 36620.93830967, 20496.80789841,
       28042.24988301, 16739.63610858, 32198.02141416, 34018.76757564,
      

In [76]:
prix_reels

array([20290, 31900, 33920, 21980, 22990, 21990, 34390, 17890, 32990,
       20490, 18990, 11990, 26290, 33490, 17490, 32490, 25990, 26590,
       26290, 22490, 17690, 34490, 36990, 38790, 18990, 32590, 28990,
       25290, 16990, 22190, 23980, 33790, 38490, 22390, 22490, 26990,
       29990, 38980, 25990, 31690, 24590, 35990, 38290, 28790, 32490,
       26190, 34490, 23790, 26299, 38290, 35990, 21490, 25990, 16990,
       31290, 35690, 22290, 46790, 15490, 28980, 21990, 21990, 33091,
       30790, 40490, 28490, 36390, 17990, 39790, 17990, 38490, 23220,
       22190, 59980, 38990, 24790, 23290, 32480, 20480, 34990, 24590,
       27790, 38980, 28490, 23990, 26590, 22190, 15480, 23390, 34490,
       52890, 29290, 27590, 27990, 26490, 34490, 32490, 15290, 35690,
       27690, 35690, 39990, 42580, 36490, 26190, 32390, 35990, 28690,
       32300, 28990, 24990, 26590, 19290, 29990, 42590, 46290, 31580,
       52190, 42590, 37990, 28590, 19290, 39390, 39990, 39690, 26990,
       40490, 38280,

Objectif : maximiser le surplus de l'acheteur, i.e, si l'acheteur à un budget R, et n contraintes, quel véhicule maximiserait son surplus


In [78]:
data_finale = df

In [79]:
data_finale['prix_predits'] = prix_prédits
data_finale['surplus_conso'] = data_finale['prix'] - data_finale['prix_predits']

In [80]:
data_finale

,prix,kilometrage,date_mise_circulation,puissance,nb_places,puissance_fiscale,critair,ptac,nb_portes,108,...,SPOTICAR ESSENTIAL,SPOTICAR PREMIUM,Ex-Auto-école,Ex-Import,Ex-Loueur,Ex-Particulier,Ex-Société,Véhicule de direction,prix_predits,surplus_conso
0,20290,75967,2019,130.0,5.0,6.0,2.0,1890.000000,5.000000,0,...,0,1,0,0,1,0,0,0,21670.754173,-1380.754173
1,31900,29066,2021,130.0,5.0,7.0,2.0,1874.729367,5.000000,0,...,0,1,0,0,1,0,0,0,29503.834510,2396.165490
2,33920,31997,2021,130.0,5.0,7.0,1.0,1874.729367,5.000000,0,...,0,1,0,0,1,0,0,0,30052.104887,3867.895113
3,21980,40952,2021,100.0,5.0,5.0,1.0,1874.729367,5.000000,0,...,0,1,0,0,1,0,0,0,21985.132637,-5.132637
4,22990,7901,2021,100.0,5.0,5.0,1.0,1874.729367,5.000000,0,...,0,1,0,0,0,1,0,0,25035.290021,-2045.290021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,33990,3254,2022,131.0,5.0,7.0,2.0,1940.000000,5.000000,0,...,0,1,0,0,1,0,0,0,32679.588528,1310.411472
596,31490,2535,2022,131.0,5.0,7.0,2.0,1770.000000,5.000000,0,...,0,1,0,1,0,0,0,0,32398.579948,-908.579948
597,37490,21164,2022,131.0,5.0,7.0,2.0,2000.000000,5.000000,0,...,0,1,0,0,1,0,0,0,35387.230371,2102.769629
598,36490,10,2022,145.0,3.0,7.0,2.0,1874.729367,4.934673,0,...,0,1,0,0,1,0,0,0,28053.564667,8436.435333


In [81]:
newdf =data_finale.loc[
    (data_finale["208"] == 1) & 
    (data_finale["Essence"]) & 
    (data_finale['Gris']) &
    (data_finale["Citadine"]) &
    (data_finale['2']) &
    (data_finale.date_mise_circulation >= 2017) 
    ]

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (2683686276.py, line 6)

In [ ]:
newdf.head()

,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes,prix_predits,surplus_conso


In [ ]:
meilleur_choix = newdf.loc[newdf['surplus_conso'] == newdf['surplus_conso'].max()]

In [ ]:
tri_meilleures = newdf.sort_values(by = 'surplus_conso', ascending = False).head()

In [ ]:
tri_meilleures

,modele,carburant,prix,kilometrage,garantie_kilometrage,boite_de_vitesse,transmission,couleur,garantie,date_mise_circulation,puissance,silhouette,nb_places,utilisation_prec,puissance_fiscale,critair,ptac,nb_portes,prix_predits,surplus_conso


In [ ]:
tri_meilleures.index

Int64Index([], dtype='int64')